In [ ]:
import os
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
!source $HOME/.cargo/env
os.environ['PATH'] += ':/root/.cargo/bin'
!echo $PATH
!pip install outlines

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2024-10-17, rust version 1.82.0 (f6e511eec 2024-10-15)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
 70.8 MiB /  70.8 MiB (100 %)  30.7 MiB/s in  2s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
  8.5 MiB /   8.5 MiB (100 %)   5.8 MiB/s in  1s ETA:  0s
info: installing component 'clippy'
info: installing component 'rust-docs'
 16.3 MiB /  16.3 MiB (100 %)   1.5 MiB/s in 12s ETA:  0s
info: installing component 'rust-std'
 26.0 MiB /  26.0 MiB (100 %)   6.0 MiB/s in  4s ETA:  0s
info: installing component 'rustc'
 70.8 MiB /  70.8 MiB (100 %)   6.7 MiB/s in 11s ETA:  0s
info: installing component 'rustfmt'
info:

In [ ]:
!pip install gigax
!pip install python-dotenv
# !pip install flash-attn
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3510591 sha256=e5d367a20fc3b39b84db08466f7d62b2ac1c5e5b12ea8a91b9c1f69746643c5a
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from dotenv import load_dotenv
from transformers import LlamaTokenizerFast, MistralForCausalLM, AutoTokenizer, AutoModelForCausalLM
from outlines import models
import bitsandbytes as bnb  # Required for 4-bit quantization
from gigax.step import NPCStepper

# Load environment variables
load_dotenv()

# Prefer HF_TOKEN from env/Colab secrets; do not hardcode
# In Colab: set a secret named HF_TOKEN in the sidebar → Secrets
# Locally: create a .env with HF_TOKEN=your_token
hf_token = os.environ.get("HF_TOKEN")


def initialize_stepper_transformers():
    tokenizer = AutoTokenizer.from_pretrained("Gigax/NPC-LLM-7B", use_auth_token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        "Gigax/NPC-LLM-7B",
        device_map="cuda",
        load_in_8bit=True,
        use_auth_token=hf_token
    )

    model.eval()  # Set model to evaluation mode after loading

    # Initialize the model wrapper
    model_wrapper = models.Transformers(model, tokenizer)

    # Initialize the stepper with the model
    stepper = NPCStepper(model=model_wrapper)
    return stepper

# Initialize stepper
stepper = initialize_stepper_transformers()

# Print confirmation that the stepper is ready
print("Model and stepper initialized. Ready to accept inputs.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model and stepper initialized. Ready to accept inputs.


In [ ]:
import time
import asyncio
import os
import re
from gigax.parse import CharacterAction
from gigax.scene import Character, Item, Location, ProtagonistCharacter, Skill, ParameterType
from flask import Flask, jsonify, request
from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig
import threading

# Configure ngrok from environment variables (set in Colab Secrets or .env)
# NGROK_AUTHTOKEN is required for authenticated tunnels
ngrok_authtoken = os.getenv("NGROK_AUTHTOKEN")
ngrok_region = os.getenv("NGROK_REGION", "us")  # valid: us, eu, ap, au, sa, jp, in
ngrok_domain = os.getenv("NGROK_DOMAIN")  # optional reserved domain

if ngrok_authtoken:
    ngrok.set_auth_token(ngrok_authtoken)

# Predefine static components outside the loop to avoid redundant computations
context = "Modern city, you do not know anyone or anything except the given NPCs and skills. If asked about anyone else you will say you do not know them."
current_location = Location(name="City Center", description="A bustling urban area.")
locations = [current_location]  # Can add more locations as needed

# Define four separate lists for each set of NPCs
npc_set_1 = [
    Character(name="Mini Mart Manager Alex", description="You are the friendly and organized manager of the Mini Mart", current_location=current_location),
    Character(name="Banker Sarah", description="You are a diligent and trustworthy banker", current_location=current_location),
    Character(name="Doctor Patel", description="You are a compassionate and knowledgeable doctor", current_location=current_location),
]

npc_set_2 = [
    Character(name="Mini Mart Manager Jamie", description="You are the friendly and organized manager of the Mini Mart", current_location=current_location),
    Character(name="Banker Raj", description="You are a diligent and trustworthy banker", current_location=current_location),
    Character(name="Doctor Huang", description="You are a compassionate and knowledgeable doctor", current_location=current_location),
]

npc_set_3 = [
    Character(name="Mini Mart Manager Leah", description="You are the friendly and organized manager of the Mini Mart", current_location=current_location),
    Character(name="Banker James", description="You are a diligent and trustworthy banker", current_location=current_location),
    Character(name="Doctor Chang", description="You are a compassionate and knowledgeable doctor", current_location=current_location),
]

npc_set_4 = [
    Character(name="Mini Mart Manager Kim", description="You are the friendly and organized manager of the Mini Mart", current_location=current_location),
    Character(name="Banker Carlos", description="You are a diligent and trustworthy banker", current_location=current_location),
    Character(name="Doctor Singh", description="You are a compassionate and knowledgeable doctor", current_location=current_location),
]

# Define items
items = [Item(name="Cash Register", description="A machine for tracking sales")]

# Define protagonist (assumed to be static)
protagonist = ProtagonistCharacter(
    name="Chris",
    description="Curious city dweller",
    current_location=current_location,
    memories=["Helped a neighbor", "Attended a city festival"],
    quests=["Get a bank loan", "Find medicine"],
    skills=[Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])],
    psychological_profile="Observant and determined",
)

app = Flask(__name__)

def BulawaAgaya(user_input, character_choice):
    try:
        if user_input.strip().lower() in {"exit", "quit"}:
            return "Exiting..."

        # Assign the relevant NPC set and character details based on the character choice
        if character_choice == 1:  # Set 1 - Alex
            npc_set = npc_set_1
            talk = {
                "name": "Mini Mart Manager Alex",
                "description": "You are the friendly and organized manager of the Mini Mart",
                "current_location": current_location,
                "memories": ["Helped set up the mini mart", "Negotiated prices with local suppliers"],
                "quests": ["Keep the store well-stocked", "Train new staff"],
                "skills": [
                    Skill(name="Manage", description="Organize store operations", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Friendly and efficient",
            }
        elif character_choice == 2:  # Set 1 - Sarah
            npc_set = npc_set_1
            talk = {
                "name": "Banker Sarah",
                "description": "You are a diligent and trustworthy banker",
                "current_location": current_location,
                "memories": ["Helped a customer save money", "Approved a loan for a local business"],
                "quests": ["Approve a new loan", "Advise customers on savings plans"],
                "skills": [
                    Skill(name="Advise", description="Provide financial advice", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Reliable and knowledgeable",
            }
        elif character_choice == 3:  # Set 1 - Patel
            npc_set = npc_set_1
            talk = {
                "name": "Doctor Patel",
                "description": "You are a compassionate and knowledgeable doctor",
                "current_location": current_location,
                "memories": ["Diagnosed a rare illness", "Helped open the local clinic"],
                "quests": ["Research a new treatment", "Mentor a medical student"],
                "skills": [
                    Skill(name="Diagnose", description="Identify a disease or ailment", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Empathetic and detail-oriented",
            }
        elif character_choice == 4:  # Set 2 - Jamie
            npc_set = npc_set_2
            talk = {
                "name": "Mini Mart Manager Jamie",
                "description": "You are the friendly and organized manager of the Mini Mart",
                "current_location": current_location,
                "memories": ["Organized the store’s grand opening", "Ensured smooth daily operations"],
                "quests": ["Restock essential items", "Improve customer service"],
                "skills": [
                    Skill(name="Manage", description="Organize store operations", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Friendly and efficient",
            }
        elif character_choice == 5:  # Set 2 - Raj
            npc_set = npc_set_2
            talk = {
                "name": "Banker Raj",
                "description": "You are a diligent and trustworthy banker",
                "current_location": current_location,
                "memories": ["Helped a customer save money", "Approved a loan for a local business"],
                "quests": ["Approve a new loan", "Advise customers on savings plans"],
                "skills": [
                    Skill(name="Advise", description="Provide financial advice", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Reliable and knowledgeable",
            }
        elif character_choice == 6:  # Set 2 - Huang
            npc_set = npc_set_2
            talk = {
                "name": "Doctor Huang",
                "description": "You are a compassionate and knowledgeable doctor",
                "current_location": current_location,
                "memories": ["Diagnosed a rare illness", "Helped open the local clinic"],
                "quests": ["Research a new treatment", "Mentor a medical student"],
                "skills": [
                    Skill(name="Diagnose", description="Identify a disease or ailment", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Empathetic and detail-oriented",
            }
        elif character_choice == 7:  # Set 3 - Leah
            npc_set = npc_set_3
            talk = {
                "name": "Mini Mart Manager Leah",
                "description": "You are the friendly and organized manager of the Mini Mart",
                "current_location": current_location,
                "memories": ["Organized the store’s grand opening", "Ensured smooth daily operations"],
                "quests": ["Restock essential items", "Improve customer service"],
                "skills": [
                    Skill(name="Manage", description="Organize store operations", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Friendly and efficient",
            }
        elif character_choice == 8:  # Set 3 - James
            npc_set = npc_set_3
            talk = {
                "name": "Banker James",
                "description": "You are a diligent and trustworthy banker",
                "current_location": current_location,
                "memories": ["Helped a customer save money", "Approved a loan for a local business"],
                "quests": ["Approve a new loan", "Advise customers on savings plans"],
                "skills": [
                    Skill(name="Advise", description="Provide financial advice", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Reliable and knowledgeable",
            }
        elif character_choice == 9:  # Set 3 - Chang
            npc_set = npc_set_3
            talk = {
                "name": "Doctor Chang",
                "description": "You are a compassionate and knowledgeable doctor",
                "current_location": current_location,
                "memories": ["Diagnosed a rare illness", "Helped open the local clinic"],
                "quests": ["Research a new treatment", "Mentor a medical student"],
                "skills": [
                    Skill(name="Diagnose", description="Identify a disease or ailment", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Empathetic and detail-oriented",
            }
        elif character_choice == 10:  # Set 4 - Kim
            npc_set = npc_set_4
            talk = {
                "name": "Mini Mart Manager Kim",
                "description": "You are the friendly and organized manager of the Mini Mart",
                "current_location": current_location,
                "memories": ["Organized the store’s grand opening", "Ensured smooth daily operations"],
                "quests": ["Restock essential items", "Improve customer service"],
                "skills": [
                    Skill(name="Manage", description="Organize store operations", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Friendly and efficient",
            }
        elif character_choice == 11:  # Set 4 - Carlos
            npc_set = npc_set_4
            talk = {
                "name": "Banker Carlos",
                "description": "You are a diligent and trustworthy banker",
                "current_location": current_location,
                "memories": ["Helped a customer save money", "Approved a loan for a local business"],
                "quests": ["Approve a new loan", "Advise customers on savings plans"],
                "skills": [
                    Skill(name="Advise", description="Provide financial advice", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Reliable and knowledgeable",
            }
        elif character_choice == 12:  # Set 4 - Singh
            npc_set = npc_set_4
            talk = {
                "name": "Doctor Singh",
                "description": "You are a compassionate and knowledgeable doctor",
                "current_location": current_location,
                "memories": ["Diagnosed a rare illness", "Helped open the local clinic"],
                "quests": ["Research a new treatment", "Mentor a medical student"],
                "skills": [
                    Skill(name="Diagnose", description="Identify a disease or ailment", parameter_types=[ParameterType.content]),
                    Skill(name="Say", description="Speak something out loud", parameter_types=[ParameterType.content])
                ],
                "psychological_profile": "Empathetic and detail-oriented",
            }
        # (Repeat similar structure for the rest of Set 2, Set 3, and Set 4 NPCs)

        # Convert selected NPC to a ProtagonistCharacter to let them perform actions
        speaker = ProtagonistCharacter(
            name=talk["name"],
            description=talk["description"],
            current_location=talk["current_location"],
            memories=talk["memories"],
            quests=talk["quests"],
            skills=talk["skills"],
            psychological_profile=talk["psychological_profile"],
        )

        # Define events where the character speaks
        events = [
            CharacterAction(
                command="Say",
                protagonist=protagonist,  # Set the selected NPC as the protagonist
                parameters=[user_input],
            )
        ]

        # Disable gradient calculations for inference
        with torch.no_grad():
            # Run the asynchronous get_action function and wait for the result
            output = asyncio.run(stepper.get_action(
                context=context,
                locations=locations,
                NPCs=npc_set,  # Pass only the relevant NPC set
                protagonist=speaker,
                items=items,
                events=events,
            ))
        # Define the regex pattern to match the first word after a colon and space
        pattern = r"(:\s)(\S+)(\s+)"

        # Use re.sub to replace the first word after the colon and space with an empty string
        output = re.sub(pattern, r"\1", str(output), count=1)  # \1 keeps the colon and the space
        # Convert the output to a string or a dictionary that can be JSON serialized
        if isinstance(output, list):
            return [str(event) for event in output]  # Convert each CharacterAction to string
        else:
            return str(output)  # Just in case it's not a list

    except KeyboardInterrupt:
        return "\nProcess interrupted by user."
    except Exception as e:
        return f"An error occurred: {e}"

    return output

@app.route("/AcchaProject", methods=["POST"])
def AcchaProject():
    alily = request.get_json()

    input_value = alily.get('input', 'No input provided')  # Get user input from the request
    character_choice = alily.get('character', 1)  # Get character choice from the request

    result = BulawaAgaya(input_value, character_choice)  # Call the BulawaAgaya function with both input and character choice

    return jsonify(result)

def main():
    print("Starting the Flask app...")
    # Optional: apply region via config if provided
    _ = PyngrokConfig(auth_token=ngrok_authtoken, region=ngrok_region)

    # Set up ngrok tunnel
    if ngrok_domain:
        tunnel = ngrok.connect(addr=5001, domain=ngrok_domain)
    else:
        tunnel = ngrok.connect(addr=5001)

    print("Public URL:", tunnel)

    # Run the Flask app in a separate thread
    thread = threading.Thread(target=lambda: app.run(host="0.0.0.0", port=5001))
    thread.start()

if __name__ == "__main__":
    main()

Starting the Flask app...
Public URL: NgrokTunnel: "https://large-likely-chicken.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
import os, requests

domain = os.getenv("NGROK_DOMAIN")
url = f"https://{domain}/AcchaProject" if domain else "http://localhost:5001/AcchaProject"

print(f"Posting to: {url}")
resp = requests.post(url, json={"input": "Hey, do you know Leah from the other store?", "character": 8})
print(resp.text)

INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 16:18:56] "POST /AcchaProject HTTP/1.1" 200 -


"Banker James: I know Leah; she's a great manager at the Mini Mart."
